### Configuration and Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import pandas as pd
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import scipy.ndimage

from neural_final_proj.core_models import CustomCNN, ResNetSmall 
from neural_final_proj.basic_utils import prepare_data_loaders, GradCAM, visualize_saliency_map
from neural_final_proj.adversarial_logic import (
    load_models_for_attack, 
    generate_adversarial_set, 
    transferability_test,
    save_example_images, 
    save_interpretability_maps,
)

class ConfigB:
    data_root = "data/Sports" 
    image_size = 64
    batch_size = 64
    num_classes = 10 
    device = torch.device("cpu") 
    
    # Model checkpoint paths (ADJUST THESE NAMES/PATHS IF NECESSARY)
    ckpt_dir = "problem_A_outputs" 
    model_a_ckpt = "CustomCNN-original.pt"
    model_b_ckpt = "ResNetSmall-original.pt" # Assuming your ResNet was named ResNetSmall
    out_dir = "problem_B_outputs"

    # Attack settings
    eps = 8 / 255.0         # L-inf constraint (approx 0.031)
    pgd_alpha = 2 / 255.0   # Step size 
    pgd_steps = 20

    # Target label index for "basketball" 
    target_class_name = "basketball" 

cfg = ConfigB()

In [3]:
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir('..')
current_folder_name = os.path.basename(os.getcwd())
os.makedirs(cfg.out_dir, exist_ok=True)

WIDTH = 19
print(f"{'Current directory':<{WIDTH}}: {current_folder_name}")
print(f"{'Device':<{WIDTH}}: {cfg.device}")
print(f"{'Using target class':<{WIDTH}}: {cfg.target_class_name}")
print(f"{'Output Location':<{WIDTH}}: {cfg.out_dir}\n")

val_loader, _, class_names, _ = prepare_data_loaders(
    cfg.data_root, cfg.image_size, cfg.batch_size
)

# Load trained models from Part A
model_a, model_b, basketball_idx = load_models_for_attack(
    cfg, CustomCNN, ResNetSmall, class_names, cfg.ckpt_dir, cfg.model_a_ckpt, cfg.model_b_ckpt
)

print(f"\nBasketball index (Target Index) : {basketball_idx}")

Current directory  : EE4745_Final_Proj
Device             : cpu
Using target class : basketball
Output Location    : problem_B_outputs

Loading Model A from problem_A_outputs\CustomCNN-original.pt
Loading Model B from problem_A_outputs\ResNetSmall-original.pt

Basketball index (Target Index) : 1


### Main Experiment Runner and Collection

In [4]:
all_results = {}
attacks_to_run = [
    ("fgsm", False, "Untargeted"),
    ("fgsm", True, "Targeted"),
    ("pgd", False, "Untargeted"),
    ("pgd", True, "Targeted"),
]
NUM_SAMPLES = 10 # Required number of examples per attack type

for attack_name, targeted, atype in attacks_to_run:
    key = f"{attack_name}_{atype}"
    print(f"\n--- Running {key} Attack ---")
    
    
    # --- Generate on Model A (Source) ---
    adv_a = generate_adversarial_set(
        model_a, val_loader, attack_name, cfg.eps, targeted, basketball_idx, num_samples=NUM_SAMPLES, 
        alpha=cfg.pgd_alpha, steps=cfg.pgd_steps
    )
    
    # Save visualizations and maps for Model A
    prefix_a = f"modelA_{key}"
    save_example_images(adv_a, class_names, os.path.join(cfg.out_dir, "examples"), prefix_a)
    for i, ex in enumerate(adv_a[:3]): # Only save maps for first 3 examples
        save_interpretability_maps(
            model_a, ex["x_clean"], ex["x_adv"], ex["y_true"], ex["y_pred_adv"], 
            class_names, os.path.join(cfg.out_dir, "interpretability"), 
            prefix=f"{prefix_a}_ex{i}", 
            GradCAM_Class=GradCAM, Saliency_Func=visualize_saliency_map
        )
    
    # Test Transfer A -> B
    transfer_a_to_b = transferability_test(adv_a, model_b, targeted, basketball_idx)


    # --- Generate on Model B (Source) ---
    adv_b = generate_adversarial_set(
        model_b, val_loader, attack_name, cfg.eps, targeted, basketball_idx, num_samples=NUM_SAMPLES, 
        alpha=cfg.pgd_alpha, steps=cfg.pgd_steps
    )
    
    # Test Transfer B -> A
    transfer_b_to_a = transferability_test(adv_b, model_a, targeted, basketball_idx)
    
    success_a = sum(int(e["success"]) for e in adv_a) / max(len(adv_a), 1)
    success_b = sum(int(e["success"]) for e in adv_b) / max(len(adv_b), 1)

    all_results[key] = {
        "Model A Success Rate": success_a,
        "Model B Success Rate": success_b,
        "Transfer A->B Rate": transfer_a_to_b,
        "Transfer B->A Rate": transfer_b_to_a,
        "Avg Linf Norm (A)": np.mean([e['linf'] for e in adv_a]),
        "Avg L2 Norm (A)": np.mean([e['l2'] for e in adv_a]),
        "Adv_A Examples": adv_a, # Keep examples for final report details
        "Adv_B Examples": adv_b,
    }

    print(f"Model A Success: {success_a:.3f} | Transfer A->B: {transfer_a_to_b:.3f}")
    print(f"Model B Success: {success_b:.3f} | Transfer B->A: {transfer_b_to_a:.3f}")


--- Running fgsm_Untargeted Attack ---
Model A Success: 0.200 | Transfer A->B: 0.400
Model B Success: 0.400 | Transfer B->A: 0.100

--- Running fgsm_Targeted Attack ---
Model A Success: 0.100 | Transfer A->B: 0.100
Model B Success: 0.300 | Transfer B->A: 0.200

--- Running pgd_Untargeted Attack ---
Model A Success: 0.200 | Transfer A->B: 0.200
Model B Success: 0.200 | Transfer B->A: 0.000

--- Running pgd_Targeted Attack ---
Model A Success: 0.200 | Transfer A->B: 0.100
Model B Success: 0.200 | Transfer B->A: 0.000


### Final Reporting Table

In [5]:
data_for_df = []
for key, res in all_results.items():
    attack_name, targeted = key.split('_')
    
    data_for_df.append({
        'Attack': attack_name.upper(),
        'Targeted': targeted.capitalize(),
        'Source Success (A)': f"{res['Model A Success Rate']:.3f}",
        'Source Success (B)': f"{res['Model B Success Rate']:.3f}",
        'Transfer A->B': f"{res['Transfer A->B Rate']:.3f}",
        'Transfer B->A': f"{res['Transfer B->A Rate']:.3f}",
        'Avg Linf Norm': f"{res['Avg Linf Norm (A)']:.5f}",
        'Avg L2 Norm': f"{res['Avg L2 Norm (A)']:.5f}",
    })

df = pd.DataFrame(data_for_df)

print(f"Experiment Settings: L_inf Epsilon = {cfg.eps:.4f}, PGD Steps = {cfg.pgd_steps}")
print(df.to_markdown(index=False))

print(f"\n\nNote: Generated images and interpretability maps are saved in the '{cfg.out_dir}' folder.")

Experiment Settings: L_inf Epsilon = 0.0314, PGD Steps = 20
| Attack   | Targeted   |   Source Success (A) |   Source Success (B) |   Transfer A->B |   Transfer B->A |   Avg Linf Norm |   Avg L2 Norm |
|:---------|:-----------|---------------------:|---------------------:|----------------:|----------------:|----------------:|--------------:|
| FGSM     | Untargeted |                  0.2 |                  0.4 |             0.4 |             0.1 |         0.03137 |       3.42709 |
| FGSM     | Targeted   |                  0.1 |                  0.3 |             0.1 |             0.2 |         0.03137 |       3.44028 |
| PGD      | Untargeted |                  0.2 |                  0.2 |             0.2 |             0   |         0.03137 |       3.30228 |
| PGD      | Targeted   |                  0.2 |                  0.2 |             0.1 |             0   |         0.03137 |       3.28777 |


Note: Generated images and interpretability maps are saved in the 'problem_B_outputs' 